In [1]:
import pandas as pd

# Replace 'your_file.csv' with the path to your file
file_path = './Synthetic-Persona-Chat/data/Synthetic-Persona-Chat_train.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dfframe
print(df.head())

                                     user 1 personas  \
0  I am 32.\nI do not want a job.\nI play video g...   
1  I am 32.\nI play video games all day.\nI still...   
2  I am 32.\nI play video games all day.\nI still...   
3  I write.\nI work at mcdonald s.\nI watch youtu...   
4  I am bald.\nI like to swim.\nMy favorite drink...   

                                     user 2 personas  \
0  My favorite drink is iced coffee.\nI have a bl...   
1  I have a ford f150.\nI like ford cars.\nMy tru...   
2  I can recite the movie young frankenstein word...   
3  I want to move.\nI don t like feeling controll...   
4  My favorite store is american eagle.\nI enjoy ...   

                         Best Generated Conversation  
0  User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...  
1  User 1: Hey, how's it going?\nUser 2: Good, I'...  
2  User 1: Hi, my name is John. What's your name?...  
3  User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...  
4  User 1: Hello!\nUser 2: Hi!\nUser 1: What do y..

In [2]:
data_list = df.to_dict(orient='records')
data_list[0:5]

[{'user 1 personas': 'I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.',
  'user 2 personas': 'My favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer.',
  'Best Generated Conversation': "User 1: Hi! I'm [user 1's name].\nUser 2: Hi [user 1's name], I'm [user 2's name].\nUser 1: What do you do for fun?\nUser 2: I like to play video games, go to the beach, and read.\nUser 1: I like to play video games too! I'm not much of a reader, though.\nUser 2: What video games do you like to play?\nUser 1: I like to play a lot of different games, but I'm really into competitive online games right now.\nUser 2: I'm not really into competitive games, I like to play more relaxing games.\nUser 1: That's cool. What kind of relaxing games do you like to play?\nUser 2: I like to play puzzle games, simulation games, and story-based games.\nUser 1: I've never b

In [3]:
data_list = [{"persona":instance["user 1 personas"],"conversation":instance["Best Generated Conversation"]} for instance in data_list]

In [4]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
add_special_tokens = {'additional_special_tokens': ['<query>', '<response>', '<latent>', '<persona>']}
tokenizer.add_special_tokens(add_special_tokens)

4

In [5]:
args = {
   "num_latent": 10,
   "num_latent2": 10,
}

In [6]:
encoded_tokens = tokenizer.encode(add_special_tokens['additional_special_tokens'])
args['bos'], args["query"], args["response"], args["latent"], args["persona"], args["eos"] = encoded_tokens
args["pad"] = 1

In [7]:
args

{'num_latent': 10,
 'num_latent2': 10,
 'bos': 0,
 'query': 50265,
 'response': 50266,
 'latent': 50267,
 'persona': 50268,
 'eos': 2,
 'pad': 1}

In [8]:
for i, instance in enumerate(data_list):
    try:
        instance["persona"] = instance["persona"].split("\n")
        instance["conversation"] = instance["conversation"].split("\n")
        instance["persona"] = [p.strip() for p in instance["persona"] if p.strip()]
        instance["conversation"] = [c.strip() for c in instance["conversation"] if c.strip()]

        instance["conversation"] = [c.replace("User 1:", "") for c in instance["conversation"]]
        instance["conversation"] = [c.replace("User 2:", "").strip() for c in instance["conversation"]]
    except Exception as e:
        print("Error in instance:", instance)
        print("Error in instance index:", i)
        print("Exception:", e)
        continue

In [106]:
data_list[510]

{'persona': ['I love playing video games.',
  'Hey there my name is jordan and i am a veterinarian.',
  'I am also a musician on the weekends.',
  'Love to read drama books.'],
 'conversation': nan}

In [108]:
data_list.pop(510)

{'persona': ['I run to relieve stress.',
  'I am learning jujitsu but am still new to it.',
  'I am taking college and hoping to be a teacher.',
  'Poker is my favorite card game.',
  'I am a huge ed sheeran fan.'],
 'conversation': ['What do you do for fun?',
  "I love to read fantasy books. I've been reading the Lord of the Rings series recently.",
  "Oh, I love the Lord of the Rings! I've read the books and seen the movies.",
  'Me too! The books are so much better than the movies, though.',
  'I agree. The books have so much more detail and world-building.',
  "What's your favorite part of the books?",
  "I love the characters. They're so well-developed and relatable.",
  "Me too! I also love the story. It's so epic and inspiring.",
  "I agree. It's one of my favorite books of all time.",
  "Me too! I'm so glad I met someone who loves the Lord of the Rings as much as I do.",
  "Me too! It's always nice to meet a fellow fan.",
  'So, what other hobbies do you have?',
  'I also like 

In [9]:
data_list[0]

{'persona': ['I am 32.',
  'I do not want a job.',
  'I play video games all day.',
  'I still live at home with my parents.'],
 'conversation': ["Hi! I'm [user 1's name].",
  "Hi [user 1's name], I'm [user 2's name].",
  'What do you do for fun?',
  'I like to play video games, go to the beach, and read.',
  "I like to play video games too! I'm not much of a reader, though.",
  'What video games do you like to play?',
  "I like to play a lot of different games, but I'm really into competitive online games right now.",
  "I'm not really into competitive games, I like to play more relaxing games.",
  "That's cool. What kind of relaxing games do you like to play?",
  'I like to play puzzle games, simulation games, and story-based games.',
  "I've never been much of a puzzle game person, but I do like simulation games and story-based games.",
  "Nice! What's your favorite simulation game?",
  "I like Stardew Valley a lot. It's a farming game, but it's also really relaxing and fun.",
  "I'

In [ ]:
for data in data_list:
    conversation = data["conversation"]
    persona = data["persona"]
    for utterance in conversation:
        if "[user 1's name]" in utterance:
            print("Found [user 1's name] in utterance:", utterance)
    break

In [ ]:
data_ids = [
    {
        "persona":[tokenizer.encode(persona) for persona in instance['persona']],
        "conversation":[tokenizer.encode(utterance) for utterance in instance['conversation']]
    }
    
    for instance in data_list
]

In [ ]:
for dat in data_ids:
    for persona in dat['persona']:
       persona.pop(0)

In [ ]:
for dat in data_ids:
    for i,utter in enumerate(dat['conversation']):
        if i % 2 == 0:
            utter[0] = args["query"]
        else:
            utter[0] = args["response"]

In [ ]:
data_ids[0]

{'persona': [[100, 95, 2162, 10, 1518, 92, 790, 4, 2],
  [100, 101, 7, 3836, 23, 5, 950, 4, 2],
  [100, 422, 10, 2335, 41227, 334, 4, 2],
  [100, 33, 10, 380, 4045, 13495, 4, 2],
  [100, 101, 602, 8, 6016, 842, 462, 16731, 4, 2]],
 'conversation': [[50265,
   30086,
   6,
   38,
   437,
   646,
   44518,
   112,
   18,
   766,
   8174,
   653,
   18,
   110,
   766,
   116,
   2],
  [50266,
   30086,
   6,
   38,
   437,
   646,
   44518,
   132,
   18,
   766,
   8174,
   85,
   18,
   2579,
   7,
   972,
   47,
   4,
   2],
  [50265, 41541, 7, 972, 47, 350, 4, 653, 32, 47, 2509, 11, 116, 2],
  [50266,
   100,
   101,
   7,
   972,
   92,
   82,
   6,
   310,
   7017,
   6664,
   354,
   14650,
   6,
   8,
   1930,
   86,
   19,
   127,
   284,
   4,
   2],
  [50265,
   11195,
   32,
   70,
   372,
   383,
   7,
   28,
   2509,
   11,
   4,
   38,
   101,
   7,
   3836,
   23,
   5,
   950,
   6,
   422,
   10,
   2335,
   41227,
   334,
   6,
   8,
   3529,
   10,
   319,
   9,
   29

In [ ]:
def get_history_windows(data, window_size=5):
    history_windows = []
    labels = []
    for i in range(len(data) - 1):
        if data[i][0] == args["query"]:
            window = []
            # add the query itself first
            window.append(data[i])

            # move backwards and collect previous turns (up to window_size)
            j = i - 1
            while j >= 0 and len(window) < window_size:
                window.insert(0, data[j])
                j -= 1
            
            history_windows.append(window)
            labels.append(data[i + 1])

    return history_windows,labels

In [ ]:
history_windows, labels = get_history_windows(data_ids[0]['conversation'], window_size=5)

In [ ]:
from itertools import chain

for l in history_windows:
    # print(l[0])
    ids= list(chain.from_iterable(l))
    print(ids)
    print(tokenizer.decode(ids))

[50265, 30086, 6, 38, 437, 646, 44518, 112, 18, 766, 8174, 653, 18, 110, 766, 116, 2]
<query> Hi, I'm [User 1's name]. What's your name?</s>
[50265, 30086, 6, 38, 437, 646, 44518, 112, 18, 766, 8174, 653, 18, 110, 766, 116, 2, 50266, 30086, 6, 38, 437, 646, 44518, 132, 18, 766, 8174, 85, 18, 2579, 7, 972, 47, 4, 2, 50265, 41541, 7, 972, 47, 350, 4, 653, 32, 47, 2509, 11, 116, 2]
<query> Hi, I'm [User 1's name]. What's your name?</s> <response> Hi, I'm [User 2's name]. It's nice to meet you.</s> <query> Nice to meet you too. What are you interested in?</s>
[50265, 30086, 6, 38, 437, 646, 44518, 112, 18, 766, 8174, 653, 18, 110, 766, 116, 2, 50266, 30086, 6, 38, 437, 646, 44518, 132, 18, 766, 8174, 85, 18, 2579, 7, 972, 47, 4, 2, 50265, 41541, 7, 972, 47, 350, 4, 653, 32, 47, 2509, 11, 116, 2, 50266, 100, 101, 7, 972, 92, 82, 6, 310, 7017, 6664, 354, 14650, 6, 8, 1930, 86, 19, 127, 284, 4, 2, 50265, 11195, 32, 70, 372, 383, 7, 28, 2509, 11, 4, 38, 101, 7, 3836, 23, 5, 950, 6, 422, 10, 23

In [ ]:
for lable in labels:
    print(tokenizer.decode(lable))

<response> Hi, I'm [User 2's name]. It's nice to meet you.</s>
<response> I like to meet new people, play ultimate frisbee, and spend time with my family.</s>
<response> That's interesting. I've never met anyone who runs a dog obedience school before.</s>
<response> That sounds like a great job. I love dogs.</s>
<response> I have a turtle named Timothy. He's my best friend.</s>
<response> That's awesome. What was its name?</s>
<response> I love the name Leonardo. It's so cool.</s>
<response> I'm glad we met. It's nice to talk to someone who likes dogs and turtles.</s>
<response> I hope we can hang out again sometime.</s>
<response> That sounds like a plan.</s>
<response> Sounds good. Bye.</s>


In [ ]:
tokenizer.decode(list(chain.from_iterable(data_ids[0]['persona'])))

'I just bought a brand new house.</s>I like to dance at the club.</s>I run a dog obedience school.</s>I have a big sweet tooth.</s>I like taking and posting selkies.</s>'

In [ ]:
# args['bos'], args["query"], args["response"], args["latent"], args["persona"], args["eos"] 

In [ ]:
instance['persona']

['I love to hang out with my friends.',
 'I love playing sports and being active.',
 'I am a 22 year old girl.',
 'I am in college studying education.',
 'I love rap music.']

In [ ]:
from itertools import chain

train_data = []

for instance in data_ids:
    history_windows, labels = get_history_windows(instance['conversation'],window_size=5)

    for window, label in zip(history_windows, labels):
        train_data.append({
            "input_ids": [args["latent"], args["persona"]] 
                          + list(chain.from_iterable(instance['persona'])) 
                          + list(chain.from_iterable(window)),
            "labels": label
        })


In [ ]:
tokenizer.decode(train_data[0]['input_ids'])

"<latent> <persona> I just bought a brand new house.</s>I like to dance at the club.</s>I run a dog obedience school.</s>I have a big sweet tooth.</s>I like taking and posting selkies.</s> <query> Hi, I'm [User 1's name]. What's your name?</s>"

In [ ]:
tokenizer.decode(train_data[0]['labels'])

"<response> Hi, I'm [User 2's name]. It's nice to meet you.</s>"

In [ ]:
import pickle

# # Save the trainset to a pickle file
# with open('trainset.pkl', 'wb') as f_train:
#     pickle.dump(trainset, f_train)

# print("trainset saved to 'trainset.pkl'")

# Save the valset to a pickle file
with open('./Synthetic-Persona-Chat/prepared_data/train_data.pkl', 'wb') as f_val:
    pickle.dump(train_data, f_val)